# Code for scraping - calling Kickstarter GraphQL end point to get text information (Story and Risks)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

## Do not run. Import df_1 

In [ ]:
# base = "/Users/ivankoh/Downloads/kickstarter data/"
# filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
# df = pd.concat(map(pd.read_csv, filepaths))
# df_1 = df.drop_duplicates(subset=['id'])
# df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
# print(len(df))
# print(len(df_1))

# # export out 
# df_1.to_csv("df_1.csv")

In [ ]:
df_1 = pd.read_csv("/Users/ivankoh/Documents/GitHub/Kickstarter-Funding-Optimization/kickstart_data_merged_with_empty.csv")

## Sample code for extracting story and risk
As there are many rows of data, we split the work among the group members so each person takes a bucket of urls to use to scrape. As such, running the below with the full dataset will take approximately 2 weeks, running 24 hours a day

In [ ]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list2 = []
risk_list2 = []

In [ ]:
# instantiate a new Session 
s = requests.Session()

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0
agents = [
         "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
         "Mozilla/5.0 (Linux; Android 5.0; Lenovo S60-a Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 YaBrowser/17.3.1.383.00 Mobile Safari/E7FBAF",
         "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 244.0.0.12.112 (iPhone12,1; iOS 15_5; en_US; en-US; scale=2.00; 828x1792; 383361019)",
         "Mozilla/5.0 (Linux; U; Android 2.2.1; en-US; GT-S5830 Build/FROYO) AppleWebKit/534.31 (KHTML, like Gecko) UCBrowser/9.0.1.275 U3/0.8.0 Mobile Safari/534.31",
         "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
         ]

# pause for 30 seconds every 5 iterations
for i in range(67115, 81405):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(30)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "operationName":"Campaign",
                    "variables":{
                        "slug": slug
                    },
                    "query": query
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(5)
            connect_try += 1
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list2.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list2.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list2.append("")
        risk_list2.append("")
        continue

# Saving Data

In [ ]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "/Users/ivankoh/Downloads/bt4222 data/44135_81404_story" #CHANGE THIS
risk_file_name = "/Users/ivankoh/Downloads/bt4222 data/44135_81404_risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list2, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list2, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

# Checking data quality

In [ ]:
print("risk:", len(risk_list2),"\n", "story:", len(story_list2))
44135 + 37270  

In [ ]:
with open('/Users/ivankoh/Downloads/bt4222 data/44135_48799_risk.json', 'r') as f:
    risk_subset = json.load(f)

In [ ]:
len(risk_subset)

In [ ]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[81405-1]['blurb'])

print("LATEST STORY IN LIST", "\n")
print(story_list2[-1])

print("LATEST RISK IN LIST", "\n")
print(risk_list2[-1])